# Kinetic Scheme Visualization

The `pyglotaran_extras.inspect.kinetic_scheme` module provides publication-quality static
visualization of kinetic decay schemes defined in pyglotaran models. It renders compartment
nodes as labeled boxes, connected by arrows annotated with rate constants.

Key features:

- **Zero extra dependencies** — uses only matplotlib (already a dependency)
- **Publication-quality export** — SVG, PNG, and PDF via `fig.savefig()`
- **Automatic layout** — hierarchical (DAGs) or force-directed (cyclic graphs)
- **Customizable** — per-node styling, color mapping, ground state bar, layout hints
- **Two entry points** — by megacomplex name(s) or by dataset name

## Basics

Let's start with the built-in simulated data from `pyglotaran` to illustrate the core API.

In [ ]:
from glotaran.testing.simulated_data.sequential_spectral_decay import SCHEME as SCHEME_SEQ
from glotaran.testing.simulated_data.parallel_spectral_decay import SCHEME as SCHEME_PAR
from pyglotaran_extras import show_kinetic_scheme, show_dataset_kinetic_scheme, KineticSchemeConfig

import matplotlib.pyplot as plt
plt.show()

### Minimal call

The simplest usage requires three arguments: the megacomplex label(s), the model, and the
parameters. The function returns a `(Figure, Axes)` tuple, just like `plt.subplots()`.

In [ ]:
fig, ax = show_kinetic_scheme(
    "megacomplex_sequential_decay",
    SCHEME_SEQ.model,
    SCHEME_SEQ.parameters,
)
fig

A parallel decay model has independent decay channels. All compartments appear side by side,
each with its own ground state decay arrow.

In [ ]:
fig, ax = show_kinetic_scheme(
    "megacomplex_parallel_decay",
    SCHEME_PAR.model,
    SCHEME_PAR.parameters,
)
fig

### Dataset-level entry point

`show_dataset_kinetic_scheme()` looks up the dataset in the model and automatically finds all
decay megacomplexes assigned to it. Non-decay megacomplexes (coherent artifact, clp-guide,
etc.) are silently skipped.

In [ ]:
fig, ax = show_dataset_kinetic_scheme(
    "dataset_1",
    SCHEME_SEQ.model,
    SCHEME_SEQ.parameters,
)
fig

### Ground state bar

By default, ground state decays are shown as downward arrows into empty space. You can opt in
to rendering a ground state bar at the bottom by setting `show_ground_state` in the config.

In [ ]:
config = KineticSchemeConfig(show_ground_state="shared")

fig, ax = show_kinetic_scheme(
    "megacomplex_sequential_decay",
    SCHEME_SEQ.model,
    SCHEME_SEQ.parameters,
    config=config,
)
fig

### Rate labels

By default, edges are annotated with the numeric rate constant value (e.g., `"500 ns⁻¹"`).
You can also display the parameter label as a prefix by setting `show_rate_labels=True`.

In [ ]:
config = KineticSchemeConfig(show_rate_labels=True, rate_unit="ps")

fig, ax = show_kinetic_scheme(
    "megacomplex_sequential_decay",
    SCHEME_SEQ.model,
    SCHEME_SEQ.parameters,
    config=config,
)
fig

### Exporting figures

Since `show_kinetic_scheme` returns a standard matplotlib `Figure`, you can export to any
format matplotlib supports.

In [ ]:
fig, ax = show_kinetic_scheme(
    "megacomplex_sequential_decay",
    SCHEME_SEQ.model,
    SCHEME_SEQ.parameters,
)

# SVG for papers (vector, infinitely scalable)
# fig.savefig("scheme.svg", bbox_inches="tight")

# PNG for presentations
# fig.savefig("scheme.png", dpi=300, bbox_inches="tight")

# PDF for LaTeX documents
# fig.savefig("scheme.pdf", bbox_inches="tight")

## Real-world example: target analysis of carotenoid triads

The examples above use simple simulated data. In practice, kinetic models are more complex.
Let's load a real target analysis model that describes the photophysics of three
carotenoid-porphyrin triads (RCG, GCRCG, RCGCR) measured in DCM.

This model has:
- Three decay megacomplexes (one per triad measurement), each with multiple k-matrices
- Coherent artifact megacomplexes (automatically skipped)
- CLP-guide megacomplexes (automatically skipped)
- Shared rate parameters across triads

In [ ]:
from glotaran.io import load_model, load_parameters

model = load_model(
    "target_rcg_gcrcg_rcgcr_refine.yml"
)
params = load_parameters(
    "target_rcg_gcrcg_rcgcr_refine-params.yml"
)

### Single megacomplex: RCG triad

Let's start by visualizing just the RCG triad's kinetic scheme. This megacomplex has 6
compartments and 12 transitions (sequential R1→R2→R3→R4 cascade, branching to a green
state G, and an additional S6 state).

In [ ]:
fig, ax = show_kinetic_scheme(
    "complex_rcg_dcm",
    model,
    params,
    title="RCG triad kinetic scheme",
)
fig

### Dataset-level: automatically skip non-decay megacomplexes

The `tas_rcg_dcm` dataset has both `artifact_rcg_dcm` (coherent artifact) and
`complex_rcg_dcm` (decay). Using `show_dataset_kinetic_scheme` automatically picks only
the decay megacomplex.

In [ ]:
fig, ax = show_dataset_kinetic_scheme(
    "tas_rcg_dcm",
    model,
    params,
    title="tas_rcg_dcm (coherent artifact auto-excluded)",
)
fig

### Multiple megacomplexes: all three triads together

You can pass a list of megacomplex labels to visualize multiple kinetic schemes in a single
figure. Compartments that appear in multiple megacomplexes are automatically merged.

In [ ]:
fig, ax = show_kinetic_scheme(
    ["complex_rcg_dcm", "complex_gcrcg_dcm", "complex_rcgcr_dcm"],
    model,
    params,
    figsize=(16, 10),
    title="All three triads",
)
fig

## Customization with `KineticSchemeConfig`

All visual aspects of the kinetic scheme can be controlled through `KineticSchemeConfig`.
Let's explore the most useful options.

### Color mapping

Use `color_mapping` to assign specific colors to compartments. This is especially useful
when comparing multiple megacomplexes to highlight which compartments correspond to the
same physical species.

In [ ]:
from pyglotaran_extras.inspect.kinetic_scheme import NodeStyleConfig

config = KineticSchemeConfig(
    color_mapping={
        "#E74C3C": ["rcg_r1", "rcg_r2", "rcg_r3", "rcg_r4"],  # red for R states
        "#27AE60": ["rcg_g"],                                    # green for G state
        "#8E44AD": ["rcg_s6"],                                   # purple for S6
    },
)

fig, ax = show_kinetic_scheme(
    "complex_rcg_dcm",
    model,
    params,
    config=config,
    title="RCG with color-coded compartments",
)
fig

### Per-node styling and display labels

Use `node_styles` to set custom display labels, colors, and sizes on a per-node basis.
The `display_label` option is useful for showing human-readable names instead of the
internal compartment names from the model.

In [ ]:
config = KineticSchemeConfig(
    node_styles={
        "rcg_r1": NodeStyleConfig(display_label="R*\u2081", facecolor="#E74C3C"),
        "rcg_r2": NodeStyleConfig(display_label="R*\u2082", facecolor="#E74C3C"),
        "rcg_r3": NodeStyleConfig(display_label="R*\u2083", facecolor="#C0392B"),
        "rcg_r4": NodeStyleConfig(display_label="R*\u2084", facecolor="#C0392B"),
        "rcg_g": NodeStyleConfig(display_label="G", facecolor="#27AE60"),
        "rcg_s6": NodeStyleConfig(display_label="S\u2086", facecolor="#8E44AD"),
    },
    show_rate_labels=True,
)

fig, ax = show_kinetic_scheme(
    "complex_rcg_dcm",
    model,
    params,
    config=config,
    title="RCG with display labels and rate parameter names",
)
fig

### Layout hints

The default hierarchical layout places nodes top-to-bottom following the transfer direction.
Within each row, you can control the left-to-right ordering using
`horizontal_layout_preference`. This is a pipe-delimited string of node labels.
Nodes listed first appear further to the left.

In [ ]:
config = KineticSchemeConfig(
    horizontal_layout_preference="species_3|species_2|species_1",
)

fig, ax = show_kinetic_scheme(
    "megacomplex_parallel_decay",
    SCHEME_PAR.model,
    SCHEME_PAR.parameters,
    config=config,
    title="Reversed horizontal ordering via layout preference",
)
fig

### Spring layout for complex graphs

For graphs with many interconnections or cycles, a force-directed (spring) layout can
sometimes produce a clearer arrangement than the hierarchical default.

In [ ]:
config = KineticSchemeConfig(
    layout_algorithm="spring",
    show_ground_state="shared",
    node_styles={
        "rcg_r1": NodeStyleConfig(display_label="R*\u2081", facecolor="#E74C3C"),
        "rcg_r2": NodeStyleConfig(display_label="R*\u2082", facecolor="#E74C3C"),
        "rcg_r3": NodeStyleConfig(display_label="R*\u2083", facecolor="#C0392B"),
        "rcg_r4": NodeStyleConfig(display_label="R*\u2084", facecolor="#C0392B"),
        "rcg_g": NodeStyleConfig(display_label="G", facecolor="#27AE60"),
        "rcg_s6": NodeStyleConfig(display_label="S\u2086", facecolor="#8E44AD"),
    },
)

fig, ax = show_kinetic_scheme(
    "complex_rcg_dcm",
    model,
    params,
    config=config,
    title="RCG with spring layout",
)
fig

### Ground state bar on a real model

The ground state bar is especially useful for target models where multiple compartments
decay to the ground state, as it provides a visual anchor for the decay endpoints.

In [ ]:
config = KineticSchemeConfig(
    show_ground_state="shared",
    node_styles={
        "rcg_r1": NodeStyleConfig(display_label="R*\u2081", facecolor="#E74C3C"),
        "rcg_r2": NodeStyleConfig(display_label="R*\u2082", facecolor="#E74C3C"),
        "rcg_r3": NodeStyleConfig(display_label="R*\u2083", facecolor="#C0392B"),
        "rcg_r4": NodeStyleConfig(display_label="R*\u2084", facecolor="#C0392B"),
        "rcg_g": NodeStyleConfig(display_label="G", facecolor="#27AE60"),
        "rcg_s6": NodeStyleConfig(display_label="S\u2086", facecolor="#8E44AD"),
    },
)

fig, ax = show_kinetic_scheme(
    "complex_rcg_dcm",
    model,
    params,
    config=config,
    title="RCG with shared ground state bar",
)
fig

### Omitting specific rate parameters

Use `omit_parameters` to exclude specific transitions. This is useful for simplifying
complex diagrams by hiding transitions that are not relevant to the discussion.

In [ ]:
config = KineticSchemeConfig(
    omit_parameters={"rates.k61", "rates.k62", "rates.k66"},
    node_styles={
        "rcg_r1": NodeStyleConfig(display_label="R*\u2081", facecolor="#E74C3C"),
        "rcg_r2": NodeStyleConfig(display_label="R*\u2082", facecolor="#E74C3C"),
        "rcg_r3": NodeStyleConfig(display_label="R*\u2083", facecolor="#C0392B"),
        "rcg_r4": NodeStyleConfig(display_label="R*\u2084", facecolor="#C0392B"),
        "rcg_g": NodeStyleConfig(display_label="G", facecolor="#27AE60"),
    },
)

fig, ax = show_kinetic_scheme(
    "complex_rcg_dcm",
    model,
    params,
    config=config,
    title="RCG without S\u2086 transitions (omitted via omit_parameters)",
)
fig

### Side-by-side comparison with subplots

Since `show_kinetic_scheme` accepts an `ax` parameter, you can embed kinetic schemes into
any matplotlib subplot layout. This is useful for comparing different models or datasets.

In [ ]:
from matplotlib.figure import Figure

fig = Figure(figsize=(16, 6))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

show_kinetic_scheme(
    "megacomplex_sequential_decay",
    SCHEME_SEQ.model,
    SCHEME_SEQ.parameters,
    ax=ax1,
    title="Sequential decay",
)

show_kinetic_scheme(
    "megacomplex_parallel_decay",
    SCHEME_PAR.model,
    SCHEME_PAR.parameters,
    ax=ax2,
    title="Parallel decay",
)

fig

## Configuration reference

Below is a summary of all `KineticSchemeConfig` options.

| Option | Type | Default | Description |
|---|---|---|---|
| `node_styles` | `dict[str, NodeStyleConfig]` | `{}` | Per-node style overrides |
| `color_mapping` | `dict[str, list[str]]` | `{}` | Map colors to node labels |
| `node_facecolor` | `str` | `"#4A90D9"` | Default node fill color |
| `node_edgecolor` | `str` | `"#2C3E50"` | Default node border color |
| `node_width` | `float` | `1.2` | Default node width |
| `node_height` | `float` | `0.6` | Default node height |
| `edge_color` | `str` | `"#555555"` | Arrow color |
| `edge_linewidth` | `float` | `1.5` | Arrow line width |
| `rate_unit` | `"ps"` \| `"ns"` | `"ns"` | Rate constant display unit |
| `rate_decimal_places` | `int \| None` | `None` | Decimal places (None = smart rounding) |
| `show_rate_labels` | `bool` | `False` | Show parameter name prefix on edges |
| `show_ground_state` | `False` \| `"shared"` \| `"per_megacomplex"` | `False` | Ground state bar mode |
| `layout_algorithm` | `"hierarchical"` \| `"spring"` \| `"manual"` | `"hierarchical"` | Layout algorithm |
| `horizontal_layout_preference` | `str \| None` | `None` | Pipe-delimited node order hint |
| `manual_positions` | `dict \| None` | `None` | User-supplied positions for manual layout |
| `horizontal_spacing` | `float` | `2.0` | Horizontal distance between nodes |
| `vertical_spacing` | `float` | `1.5` | Vertical distance between layers |
| `ground_state_offset` | `float` | `1.2` | Vertical offset for ground state |
| `figsize` | `tuple[float, float]` | `(10.0, 8.0)` | Default figure size |
| `title` | `str \| None` | `None` | Plot title |
| `omit_parameters` | `set[str]` | `set()` | Parameter labels to exclude |